In [1]:
!export HF_HOME=../hf_cache

In [2]:
!git config --global credential.helper store
!pip install accelerate
!git config --global user.name "Neelectric"
!git config --global user.email "Neel.R@web.de"

In [3]:
import wandb
from wandb import AlertLevel
# wandb.init(project="biollama_v2", # the project I am working on
#            tags=["hf_sft", "BioLlamaV2"]) # the Hyperparameters I want to keep track of

# wandb.alert(
#     title="Initialising training run",
#     text=f"We have started training",
#     level=AlertLevel.WARN,
#     wait_duration=300,
# )

In [4]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("imported...")
print(transformers.__version__)

/opt/miniconda3/envs/biollama/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


imported...
4.39.3


Next, we pull the huggingface token from the local config. Remember to make a copy of config_default.yml, name it config.yml and add your huggingface token. config.yml is in the gitignore so unless you misspell config.yml, it should not get pushed to github.

In [5]:
# token = input(f"Enter token: ")
from box import Box
with open("config.yml", "r") as f:
    config = Box.from_yaml(f.read())
token = config.secrets.hf_token

We also need to verfiy whether the GPU supports bfloat16 (less precision but more range). If not (for example if running on TITAN GPUs), we use float16

In [6]:
if torch.cuda.is_bf16_supported():
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16
torch_dtype = torch.bfloat16
print(f"Using dtype: {torch_dtype}")

Using dtype: torch.bfloat16


In [7]:
# from transformers import AutoModel
# medcpt_path = "ncbi/MedCPT-Article-Encoder"
# document_tokenizer = AutoTokenizer.from_pretrained(medcpt_path)
# document_model = AutoModel.from_pretrained(medcpt_path).to('cuda')

In [8]:
print("started script")

llama_path = "h2oai/h2ogpt-4096-llama2-7b-chat"
# llama_path = "meta-llama/Llama-2-7b-chat-hf"
# llama_path = "TheBloke/Llama-2-7B-Chat-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(llama_path, 
                                          token = token,
                                          cache_dir = "../hf_cache/")
model = AutoModelForCausalLM.from_pretrained(llama_path, 
                                             token = token,
                                            #  device_map = "auto",
                                            device_map = "cuda:1",
                                             torch_dtype = torch_dtype,
                                             cache_dir = "../hf_cache/")

print("model loaded!")

started script


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.85s/it]

model loaded!


In [9]:
from time import time
max_new_tokens = 150
prompt = "In the era of generative AI, "
medmcqa2 = """
Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
"""
tokenized_prompt = tokenizer.encode(medmcqa2, return_tensors = "pt").to('cuda:1')
time_before = time()
raw_output = model.generate(tokenized_prompt,
                            max_new_tokens = max_new_tokens,
                            temperature = 0.01)
time_after = time()
time_taken = time_after - time_before
untokenized_output = tokenizer.decode(raw_output[0], skip_special_tokens = True)
num_generated = len(raw_output[0]) - len(tokenized_prompt[0])
print(untokenized_output)
print(f"newly generated {num_generated}")
print(f"{num_generated / time_taken} t/s")


Question.
Low insulin to glucagon ratio is not seen in:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer. 
(D) Ketogenesis

Explanation:
A low insulin to glucagon ratio is typically seen in states of ketosis, where the body is relying on ketones for energy instead of glucose. Glucagon is a hormone that raises blood glucose levels by stimulating the breakdown of glycogen and fatty acids, and it is inversely related to insulin. Therefore, a low insulin to glucagon ratio indicates that the body is in a state of ketosis, where ketones are being used as the primary source of energy. Glycogen synthesis, glycogen breakdown,
newly generated 150
30.138704158394212 t/s


In [10]:
import os
path = os.path.abspath(os.path.join(os.getcwd(), os.pardir)) + "/pubmed_cleaned/"
print(path)
files = os.listdir(path)
print(files[:5])
# load in the first file and print first 20 lines
# with open(path + files[0], "r") as f:
#     lines = f.readlines()
#     for line in lines[:20]:
#         print(line)
from datasets import load_dataset
print(path + files[0])
dataset = load_dataset("text", 
                       cache_dir = "../hf_cache/",
                       data_files = path + files[0], 
                       split = "train")
# dataset = load_dataset("imdb", split="train")

/nfs/primary/pubmed_cleaned/
['abs_1_14.tsv', 'abs_1_15.tsv', 'abs_1_23.tsv', 'abs_1_0.tsv', 'abs_1_30.tsv']
/nfs/primary/pubmed_cleaned/abs_1_14.tsv


In [11]:
dataset

Dataset({
    features: ['text'],
    num_rows: 67773
})

In [12]:
def param_count(m):
    params = sum([p.numel() for p in m.parameters()])/1_000_000
    trainable_params = sum([p.numel() for p in m.parameters() if p.requires_grad])/1_000_000
    print(f"Total params: {params:.2f}M, Trainable: {trainable_params:.2f}M")
    return params, trainable_params

for param in model.parameters(): 
    param.requires_grad = True
model.model.embed_tokens.weight.requires_grad_(False)
params, trainable_params = param_count(model)

Total params: 6738.42M, Trainable: 6607.34M


In [13]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# batch_size = 2
# total_num_steps = 50
# output_dir = "../"
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     # per_device_train_batch_size=batch_size,
#     # per_device_eval_batch_size=batch_size//2,
#     # bf16=False,
#     learning_rate=2e-4,
#     lr_scheduler_type="cosine",
#     warmup_steps=total_num_steps // 10,
#     num_train_epochs=1,
#     # max_steps = -1
#     gradient_accumulation_steps=1,
#     gradient_checkpointing=True,
#     # evaluation_strategy="steps",
#     # eval_steps=total_num_steps // 6,
#     # logging strategies
#     logging_dir=f"{output_dir}/logs",
#     logging_strategy="steps",
#     logging_steps=1,
#     save_strategy="epoch", #changed to epoch so we save every epoch i guess?
#     save_total_limit=1,
# )

In [14]:
# def create_prompt(row):
#     text = row["text"]
#     return text
# # create_prompt(dataset[0])

In [15]:
# train_dataset = dataset["text"]

In [16]:
# print(train_dataset[0])

In [17]:
# trainer = SFTTrainer(
#     model,
#     train_dataset=train_dataset,
#     dataset_text_field="text",
#     # eval_dataset=test_dataset,
#     packing=True,
#     max_seq_length=512,
#     args=training_args,
#     formatting_func=create_prompt,
#     # compute_metrics=token_accuracy,
# )

In [18]:
# #set llama model config use_cache to false!!!
# trainer.train()
# # wandb.finish()

In [19]:
# trainer.save_model(output_dir)
# #print contents of output_dir
# !ls -l $output_dir
# #print full path of output_dir
# # !pwd $output_dir

In [20]:
# import torch
# from torch.utils.data import Dataset, DataLoader

# class TextDataset(Dataset):
#     def __init__(self, data, tokenizer):
#         self.data = data
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         text = self.data[idx]['text']
#         inputs = self.tokenizer(text, 
#                                 return_tensors="pt", 
#                                 padding=True, 
#                                 max_length=10,
#                                 truncation=True)
#         return inputs
# text_dataset = TextDataset(dataset, tokenizer)
# dataloader = DataLoader(text_dataset, batch_size=1, shuffle=True)

In [21]:
import torch.optim as optim

def train(model, dataloader, epochs=5, lr=5e-5):
    device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
    # model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=lr)
    print("starting training")

    for epoch in range(epochs):
        model.train()
        for item in dataset:
            text = item["text"]
            inputs = tokenizer(text, return_tensors="pt", padding=True, max_length=10, truncation=True)
            # print(batch)
            inputs = inputs.to(device)
            print(inputs.input_ids.shape)
            print(inputs)
            outputs = model(**inputs, labels=inputs.input_ids)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

In [22]:
dataset[0]["text"]

'\tTwenty male patients with primary hypertriglyceridemia were treated for 4 weeks with daily supplements (15 g) of oil, which provided approximately 6 g of polyunsaturated fatty acid (PUFA) either of fish or of vegetable origin. Total plasma cholesterol concentrations were unaffected, but both types of supplement increased high density lipoprotein-3 (HDL3) cholesterol concentrations. The fish, but not the vegetable, oil supplement led to a decrease in plasma triglyceride concentrations. Very low density lipoprotein (VLDL), fatty acid composition, and VLDL triglyceride kinetics were subsequently studied in five patients (four male, one female) before and after 4 weeks of therapy with 15 g of the same fish oil. The fish oil led to increases in the proportion of eicosapentaenoic acid in both the VLDL triglyceride and phospholipid fractions, but the increase was greater in the latter. In contrast, the proportion of docosahexanoic acid was increased only in the VLDL triglycerides. The decr

In [24]:
tokenizer.pad_token = tokenizer.eos_token
train(model, "dataloader", epochs=5, lr=5e-5)

starting training
torch.Size([1, 10])
{'input_ids': tensor([[    1, 29871,    12, 27418,  6478, 14263, 22069,   411,  7601, 11266]],
       device='cuda:1'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:1')}


OutOfMemoryError: CUDA out of memory. Tried to allocate 86.00 MiB. GPU 1 has a total capacity of 47.54 GiB of which 7.62 MiB is free. Process 59195 has 47.53 GiB memory in use. Of the allocated memory 46.30 GiB is allocated by PyTorch, and 59.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)